In [1]:
# Данные по книгам Livelib

import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = 'https://www.livelib.ru/books/movers-and-shakers/listview/biglist/~{}'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

all_books = []
book_counter = 0

# Парсинг первых 10 страниц
for page in range(1, 4):
    url = base_url.format(page)
    print(f'Парсинг страницы {page}...')

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Находим все элементы книг
        books = soup.select('li.book-item__item')

        for book in books:
            book_counter += 1

            # Извлечение данных
            title = book.select_one('a.book-item__title').text.strip() if book.select_one('a.book-item__title') else 'Название не указано'
            author = book.select_one('a.book-item__author').text.strip() if book.select_one('a.book-item__author') else 'Автор не указан'
            rating = book.select_one('div.book-item__rating').text.strip() if book.select_one('div.book-item__rating') else '0'

            # Статус книги (хит, новинка и т.д.)
            status_tag = book.select_one('a.book-item__status')
            status = status_tag['title'] if status_tag else ''

            # Ссылка на обложку
            img_tag = book.select_one('img[data-pagespeed-lazy-src]')
            image_url = img_tag['data-pagespeed-lazy-src'] if img_tag else ''

            # Ссылка на книгу
            book_url = book.select_one('a.book-item__link')['href'] if book.select_one('a.book-item__link') else ''

            # Сбор информации о рецензиях
            reviews_block = book.select_one('div.book-item__menu-block')
            reviews_count = '0'
            if reviews_block:
                reviews_link = reviews_block.find('a', string='Поделиться')
                if reviews_link:
                    reviews_count = reviews_link.find_next_sibling('a').text.strip() if reviews_link.find_next_sibling('a') else '0'

            all_books.append({
                '№': book_counter,
                'Название': title,
                'Автор': author,
                'Рейтинг': rating.replace(',', '.'),
                'Статус': status



            })

    except Exception as e:
        print(f'Ошибка на странице {page}: {str(e)}')
        continue

# Сохранение в CSV
if all_books:
    df = pd.DataFrame(all_books)
    df.to_csv('livelib_books.csv', index=False, encoding='utf-8-sig')
    print(f'Сохранено {len(df)} книг')

Парсинг страницы 1...
Парсинг страницы 2...
Парсинг страницы 3...
Сохранено 60 книг
